In [1]:
import torch

inputs = torch.tensor(
[[0.43, 0.15, 0.89], # Your (x^1)
[0.55, 0.87, 0.66], # journey (x^2)
[0.57, 0.85, 0.64], # starts (x^3)
[0.22, 0.58, 0.33], # with (x^4)
[0.77, 0.25, 0.10], # one (x^5)
[0.05, 0.80, 0.55]] # step (x^6)
)

Computing Attention Scores



In [2]:
query = inputs[1]
attn_scores = torch.empty(inputs.shape[0])
for i in range(inputs.shape[0]):
    attn_scores[i] = torch.dot(query, inputs[i])

print(attn_scores)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


Attention Scores Normalization

In [3]:
attn_scores = torch.softmax(attn_scores, dim=0)
print(attn_scores)

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])


Computing Context Vector


In [4]:
query=inputs[1]
context_vector = torch.zeros(query.shape)

for i, x_i in enumerate(inputs):
    context_vector += attn_scores[i]*x_i
    print(context_vector)

print(context_vector)


tensor([0.0596, 0.0208, 0.1233])
tensor([0.1904, 0.2277, 0.2803])
tensor([0.3234, 0.4260, 0.4296])
tensor([0.3507, 0.4979, 0.4705])
tensor([0.4340, 0.5250, 0.4813])
tensor([0.4419, 0.6515, 0.5683])
tensor([0.4419, 0.6515, 0.5683])


Computing All Context Vectors

In [5]:
attention_scores = torch.empty(6, 6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attention_scores[i ,j] = torch.dot(x_i, x_j)

print(attention_scores)

# or attention_scores = inputs @ inputs.T

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [6]:
attention_scores = torch.softmax(attention_scores, dim=1)
print(attention_scores)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [7]:
all_context_vectors = attention_scores @ inputs
print(all_context_vectors)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


Self-attention with trainable weights

In [8]:
x_2 = inputs[1]
d_in = 3
d_out = 2

In [9]:
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [10]:
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

print(query_2)

tensor([0.9721, 0.9499])


In [11]:
keys = inputs @ W_query
value = inputs @ W_value

In [12]:
keys_2 = keys[1]
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.8474)


In [13]:
attn_scores_2 = query_2 @ keys.T
print(attn_scores_2)

tensor([1.2819, 1.8474, 1.8442, 0.9657, 1.2682, 1.0658])


Normalizing Attention Scores

In [14]:
d_k = keys.shape[-1]
attn_scores_2 = torch.softmax(attn_scores_2 / d_k ** 0.5, dim=-1)
print(attn_scores_2)

tensor([0.1509, 0.2250, 0.2245, 0.1206, 0.1494, 0.1295])


In [15]:
context_vector_2 = attn_scores_2 @ value
print(context_vector_2)

tensor([0.4177, 0.4786])


Self-attention Class

In [16]:
import torch.nn as nn

class SelfAttention(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        values = self.W_value(x)
        queries = self.W_query(x)
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim=-1)
        context_vectors = attn_weights @ values

        return context_vectors

In [17]:
sa1 = SelfAttention(3, 2)

print(sa1(inputs))

tensor([[-0.3746, -0.0968],
        [-0.3718, -0.0981],
        [-0.3716, -0.0982],
        [-0.3750, -0.0959],
        [-0.3692, -0.0991],
        [-0.3770, -0.0951]], grad_fn=<MmBackward0>)


Causal Attention Mask

In [121]:
import torch.nn as nn

class SelfAttention_test(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        values = self.W_value(x)
        queries = self.W_query(x)
        attn_scores = queries @ keys.T
        context_length = attn_scores.shape[0]

        mask_simple = torch.triu(torch.ones(context_length, context_length), diagonal=1)
        masked = attn_scores.masked_fill(mask_simple.bool(), -torch.inf)

        attn_weights = torch.softmax(masked / keys.shape[-1] ** 0.5, dim=-1)
        print(attn_weights)

        #apply dropout
        dropout = torch.nn.Dropout(0.5)
        attn_weights = dropout(attn_weights)
        print(attn_weights)
        #rescale remaining scores
        attn_weights = 2 * attn_weights
        print(attn_weights)

        context_vectors = attn_weights @ values

        return context_vectors

In [119]:
sa1 = SelfAttention(3, 2)

print(sa1(inputs))

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5246, 0.4754, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3544, 0.3219, 0.3237, 0.0000, 0.0000, 0.0000],
        [0.2582, 0.2461, 0.2470, 0.2487, 0.0000, 0.0000],
        [0.2029, 0.1970, 0.1973, 0.1982, 0.2047, 0.0000],
        [0.1740, 0.1617, 0.1625, 0.1642, 0.1813, 0.1563]],
       grad_fn=<SoftmaxBackward0>)
tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.9508, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7088, 0.6438, 0.6474, 0.0000, 0.0000, 0.0000],
        [0.5164, 0.4922, 0.0000, 0.4974, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.3946, 0.3963, 0.4095, 0.0000],
        [0.0000, 0.0000, 0.3250, 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)
tensor([[4.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.9016, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.4176, 1.2875, 1.2949, 0.0000, 0.0000, 0.0000],
        [1.0328, 0.9845, 0.0000, 0.9948, 0.0000, 0.0000],
    

In [122]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)

torch.Size([2, 6, 3])


In [141]:
class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout) 
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) 

    def forward(self, x):
        b, num_tokens, d_in = x.shape #b is batch dimension
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2) # Changed transpose
        attn_scores.masked_fill_(self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)  # type: ignore # `:num_tokens` to account for cases where the number of tokens in the batch is smaller than the supported context_size
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights) #rescaling is implicitly done

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(41)

context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.5)

context_vecs = ca(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[ 0.0000,  0.0000],
         [-0.1767,  0.2832],
         [ 0.0045,  0.4318],
         [ 0.0044,  0.5068],
         [-0.0743,  0.2544],
         [ 0.0565,  0.5107]],

        [[-0.3502,  0.5613],
         [-0.1767,  0.2832],
         [-0.0984,  1.0560],
         [ 0.0077,  0.8303],
         [ 0.1369,  0.6210],
         [-0.0064,  0.7238]]], grad_fn=<UnsafeViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


Multi-head Attention

In [145]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList([CausalAttention(d_in, d_out, context_length, dropout) for _ in range(num_heads)])
    
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)
    

In [147]:
context_length=batch.shape[1]
d_in = 3 
d_out = 2
multihead = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, 2)
context_vectors = multihead(batch)
print(context_vectors)

tensor([[[-0.1203, -0.2195, -0.0268, -0.1082],
         [ 0.0471, -0.0643,  0.1722, -0.0823],
         [ 0.1003, -0.0073,  0.2359, -0.0722],
         [ 0.1176,  0.0046,  0.2409, -0.0621],
         [ 0.0989,  0.0860,  0.2196, -0.0409],
         [ 0.1241,  0.0445,  0.2401, -0.0471]],

        [[-0.1203, -0.2195, -0.0268, -0.1082],
         [ 0.0471, -0.0643,  0.1722, -0.0823],
         [ 0.1003, -0.0073,  0.2359, -0.0722],
         [ 0.1176,  0.0046,  0.2409, -0.0621],
         [ 0.0989,  0.0860,  0.2196, -0.0409],
         [ 0.1241,  0.0445,  0.2401, -0.0471]]], grad_fn=<CatBackward0>)


In [154]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens] # type: ignore

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

torch.manual_seed(12)

batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.5683,  0.3132],
         [-0.6078,  0.2420],
         [-0.6191,  0.2256],
         [-0.6091,  0.2322],
         [-0.5941,  0.3001],
         [-0.5991,  0.2586]],

        [[-0.5683,  0.3132],
         [-0.6078,  0.2420],
         [-0.6191,  0.2256],
         [-0.6091,  0.2322],
         [-0.5941,  0.3001],
         [-0.5991,  0.2586]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])
